An attempt of Adversarial Neuron Perturbation

code references: \
https://www.kaggle.com/code/itsuki9180/introducing-adversarial-weight-perturbation-awp 

In [1]:
import torch
import torchvision

from torchsummary import summary

import numpy as np

In [2]:
torch.cuda.is_available()

True

In [3]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [44]:
res18 = torchvision.models.resnet18().cuda() if torch.cuda.is_available() else torchvision.models.resnet18()

In [45]:
# check if the parameters are in cuda
next(res18.parameters()).is_cuda

True

In [46]:
res18.load_state_dict(torch.load(f'saved_models/ResNet18-CIFAR10-Epoch-100.pth'))

C:\Users\Kevvol\AppData\Local\Temp\ipykernel_32072\2594117609.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  res18.load_state_dict(torch.load(f'saved_models/ResNet18-CI

<All keys matched successfully>

In [47]:
# print out named params
for name, param in res18.named_parameters():
    print(name)
    print(param.shape)

conv1.weight
torch.Size([64, 3, 7, 7])
bn1.weight
torch.Size([64])
bn1.bias
torch.Size([64])
layer1.0.conv1.weight
torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight
torch.Size([64])
layer1.0.bn1.bias
torch.Size([64])
layer1.0.conv2.weight
torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight
torch.Size([64])
layer1.0.bn2.bias
torch.Size([64])
layer1.1.conv1.weight
torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight
torch.Size([64])
layer1.1.bn1.bias
torch.Size([64])
layer1.1.conv2.weight
torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight
torch.Size([64])
layer1.1.bn2.bias
torch.Size([64])
layer2.0.conv1.weight
torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight
torch.Size([128])
layer2.0.bn1.bias
torch.Size([128])
layer2.0.conv2.weight
torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight
torch.Size([128])
layer2.0.bn2.bias
torch.Size([128])
layer2.0.downsample.0.weight
torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight
torch.Size([128])
layer2.0.downsample.1.bias
torch.Size([128])
layer2.1.conv1.weight
tor

In [48]:
# all params should be either weights or biases
for name, param in res18.named_parameters():
    if not name.endswith('weight') and not name.endswith('bias'):
        # should never be called
        print(name)

In [51]:
kha = torch.ones_like(param)

In [52]:
kha.uniform_(-0.1, 0.1)

tensor([ 9.7781e-02, -6.1204e-02, -8.0450e-02,  3.8314e-02,  9.1009e-02,
         3.5940e-02, -7.3964e-02, -4.5669e-02, -9.0166e-02, -3.4613e-02,
        -2.5263e-02,  2.6935e-02, -4.2742e-02,  1.3043e-02,  2.8988e-02,
        -1.3944e-02,  5.1191e-02, -6.4766e-03, -8.1087e-02,  1.2932e-02,
         3.8977e-02,  5.2983e-02,  3.0861e-03,  6.5806e-02,  8.7392e-02,
         9.1452e-02,  1.5232e-02,  5.8878e-03, -1.1795e-02,  2.6979e-02,
        -5.7707e-02,  2.4057e-02, -3.8078e-02,  8.1571e-02,  2.7058e-02,
         4.2510e-02,  8.5199e-02,  7.3530e-02,  2.1589e-03, -1.5127e-02,
         8.4344e-02, -5.8487e-02, -6.7192e-02,  3.5748e-02,  2.6120e-02,
        -1.2447e-03,  2.0210e-02,  4.7055e-02,  2.9201e-02,  5.5800e-02,
        -9.2774e-02,  2.5119e-02,  9.1138e-03, -7.6144e-02, -3.8740e-02,
         7.0275e-02,  1.1925e-02, -5.3659e-02, -9.8320e-03, -9.1633e-02,
         7.4195e-02, -5.5235e-02, -1.8153e-02,  5.5848e-02, -4.5039e-02,
        -4.8014e-02,  2.9090e-02,  1.9356e-02,  4.9

In [53]:
kha

tensor([ 9.7781e-02, -6.1204e-02, -8.0450e-02,  3.8314e-02,  9.1009e-02,
         3.5940e-02, -7.3964e-02, -4.5669e-02, -9.0166e-02, -3.4613e-02,
        -2.5263e-02,  2.6935e-02, -4.2742e-02,  1.3043e-02,  2.8988e-02,
        -1.3944e-02,  5.1191e-02, -6.4766e-03, -8.1087e-02,  1.2932e-02,
         3.8977e-02,  5.2983e-02,  3.0861e-03,  6.5806e-02,  8.7392e-02,
         9.1452e-02,  1.5232e-02,  5.8878e-03, -1.1795e-02,  2.6979e-02,
        -5.7707e-02,  2.4057e-02, -3.8078e-02,  8.1571e-02,  2.7058e-02,
         4.2510e-02,  8.5199e-02,  7.3530e-02,  2.1589e-03, -1.5127e-02,
         8.4344e-02, -5.8487e-02, -6.7192e-02,  3.5748e-02,  2.6120e-02,
        -1.2447e-03,  2.0210e-02,  4.7055e-02,  2.9201e-02,  5.5800e-02,
        -9.2774e-02,  2.5119e-02,  9.1138e-03, -7.6144e-02, -3.8740e-02,
         7.0275e-02,  1.1925e-02, -5.3659e-02, -9.8320e-03, -9.1633e-02,
         7.4195e-02, -5.5235e-02, -1.8153e-02,  5.5848e-02, -4.5039e-02,
        -4.8014e-02,  2.9090e-02,  1.9356e-02,  4.9

In [56]:
kha.requires_grad = True

In [57]:
loss_fn = torch.nn.CrossEntropyLoss()

In [58]:
import torchvision.transforms as transforms

cifar10_train = torchvision.datasets.CIFAR10('datasets/cifar_10', download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=10, shuffle=True, num_workers=4)

Files already downloaded and verified


In [59]:
kha.grad

In [60]:
param = param * (1 + kha)

In [61]:
for inputs, label in train_loader:
    inputs, label = inputs.to(device), label.to(device)
    
    pred = res18(inputs)
    loss = loss_fn(pred,label)
    loss.backward()
    # just 1 cycle and break
    break

In [62]:
kha.requires_grad

True

In [64]:
kha.grad is None

True